In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import logging
import pytorch_lightning as pl
import warnings

warnings.filterwarnings('ignore')
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

## Data load

In [2]:
! mkdir -p data
! curl -OL https://storage.yandexcloud.net/di-datasets/age-prediction-nti-sbebank-2019.zip
! unzip -j -o age-prediction-nti-sbebank-2019.zip 'data/*.csv' -d data
! mv age-prediction-nti-sbebank-2019.zip data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  239M  100  239M    0     0  55.1M      0  0:00:04  0:00:04 --:--:-- 55.1M
Archive:  age-prediction-nti-sbebank-2019.zip
  inflating: data/test.csv           
  inflating: data/small_group_description.csv  
  inflating: data/train_target.csv   
  inflating: data/transactions_train.csv  
  inflating: data/transactions_test.csv  


## Data Preproccessing

In [3]:
import os
import pandas as pd

data_path = 'data/'

source_data = pd.read_csv(os.path.join(data_path, 'transactions_train.csv'))
source_data.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [4]:
from dltranz.data_preprocessing import PandasDataPreprocessor

preprocessor = PandasDataPreprocessor(
    col_id='client_id',
    cols_event_time='trans_date',
    time_transformation='float',
    cols_category=["trans_date", "small_group"],
    cols_log_norm=["amount_rur"],
    cols_identity=[],
    print_dataset_info=False,
)

In [5]:
%%time

dataset = preprocessor.fit_transform(source_data)

CPU times: user 1min 6s, sys: 15.5 s, total: 1min 22s
Wall time: 1min 22s


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2, random_state=42)

(len(train), len(test))

(24000, 6000)

## Embedding training

Model training in our framework organised via pytorch-lightning (pl) framework.
The key parts of neural networks training in pl are: 

    * model (`pytorch_lightning.LightningModule`)
    * data loader (`torch.utils.data.DataLoader`)
    * trainer (`pytorch_lightning.Trainer`)
    
For futher details check https://pytorchlightning.ai/

### Model definition

In [7]:
from dltranz.seq_encoder import SequenceEncoder
from dltranz.models import Head
from dltranz.lightning_modules.emb_module import EmbModule

seq_encoder = SequenceEncoder(
    category_features=preprocessor.get_category_sizes(),
    numeric_features=["amount_rur"],
    trx_embedding_noize=0.003
)

head = Head(input_size=seq_encoder.embedding_size, use_norm_encoder=True)

model = EmbModule(seq_encoder=seq_encoder, head=head)

### Data module

In [8]:
from dltranz.data_load.data_module.emb_data_module import EmbeddingDatasetFactory

dlf = EmbeddingDatasetFactory(
    min_seq_len=25,
    seq_split_strategy='SampleSlices',
    category_names = model.seq_encoder.category_names,
    category_max_size = model.seq_encoder.category_max_size,
    split_count=5,
    split_cnt_min=25,
    split_cnt_max=200,
)

### Trainer

In [9]:
import torch
import pytorch_lightning as pl

import logging
# logging.getLogger("lightning").addHandler(logging.NullHandler())
# logging.getLogger("lightning").propagate = False

trainer = pl.Trainer(
#    progress_bar_refresh_rate=0,
    max_epochs=150,
    gpus=1 if torch.cuda.is_available() else 0
)

### Training 

In [12]:
train_dl = dlf.train_data_loader(train, num_workers=16, batch_size=256)
trainer.fit(model, train_dl)

Training: 0it [00:00, ?it/s]

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/dmitri/.local/share/virtualenvs/pytorch-lifestream-hd3T5skF/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/dmitri/.local/share/virtualenvs/pytorch-lifestream-hd3T5skF/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 40, in fetch
    return self.collate_fn(data)
  File "/home/dmitri/.local/share/virtualenvs/pytorch-lifestream-hd3T5skF/lib/python3.8/site-packages/pytorch_lightning/utilities/auto_restart.py", line 474, in _capture_metadata_collate
    data = default_collate(samples)
  File "/home/dmitri/pytorch-lifestream/dltranz/data_load/data_module/coles_data_module.py", line 42, in coles_collate_fn
    padded_batch = collate_fn(batch)
  File "/home/dmitri/pytorch-lifestream/dltranz/metric_learn/dataset/__init__.py", line 22, in _func
    batch = functools.reduce(

## Inference 

In [12]:
# embedding inference

train_dl = dlf.inference_data_loader(train, num_workers=0, batch_size=256)
train_embeds = trainer.predict(model, train_dl)

test_dl = dlf.inference_data_loader(test, num_workers=0, batch_size=256)
test_embeds = trainer.predict(model, test_dl)

train_embeds.shape, test_embeds.shape

24000it [00:01, 21719.33it/s]
6000it [00:00, 31278.41it/s]                                                                                                            


((24000, 512), (6000, 512))

In [13]:
# join target and embeddings

df_target = pd.read_csv(os.path.join(data_path, 'train_target.csv'))
df_target = df_target.set_index('client_id')
df_target.rename(columns={"bins": "target"}, inplace=True)

train_df = pd.DataFrame(data=train_embeds, columns=[f'embed_{i}' for i in range(train_embeds.shape[1])])
train_df['client_id'] = [x['client_id'] for x in train]
train_df = train_df.merge(df_target, how='left', on='client_id')

test_df = pd.DataFrame(data=test_embeds, columns=[f'embed_{i}' for i in range(test_embeds.shape[1])])
test_df['client_id'] = [x['client_id'] for x in test]
test_df = test_df.merge(df_target, how='left', on='client_id')

print(train_df.shape, test_df.shape)

(24000, 514) (6000, 514)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_504,embed_505,embed_506,embed_507,embed_508,embed_509,embed_510,embed_511,client_id,target
0,0.344823,0.340152,0.231864,-0.789117,-0.013289,-0.056129,-0.988241,-0.010464,-0.064898,-0.029179,...,0.342959,0.040367,0.253053,0.712581,-0.148498,0.016645,-0.124844,-0.078120,36253,1
1,0.276711,0.492666,0.781279,-0.824952,0.020340,-0.014695,-0.891105,-0.047485,0.063514,0.170822,...,0.372329,0.049816,0.346733,0.465750,-0.080580,0.011563,-0.040877,-0.028394,396,2


Obtained embeddings can be used as features for model training

For example:

In [14]:
from sklearn.linear_model import LogisticRegression

embed_columns = [x for x in train_df.columns if x.startswith('embed')]
x_train, y_train = train_df[embed_columns], train_df['target']
x_test, y_test = test_df[embed_columns], test_df['target']

clf = LogisticRegression()
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.6296666666666667